# Problem 209: Circular Logic

A $k$-input <strong>binary truth table</strong> is a map from $k$ input bits (binary digits, $0$ [false] or $1$ [true]) to $1$ output bit. For example, the $2$-input binary truth tables for the logical $\mathbin{\text{AND}}$ and $\mathbin{\text{XOR}}$ functions are:

<div style="float:left;margin:10px 50px;text-align:center;">
<table class="grid"><tr><th style="width:50px;">x</th>
<th style="width:50px;">y</th>
<th>x AND y</th></tr>
<tr><td align="center">0</td><td align="center">0</td><td align="center">0</td></tr><tr><td align="center">0</td><td align="center">1</td><td align="center">0</td></tr><tr><td align="center">1</td><td align="center">0</td><td align="center">0</td></tr><tr><td align="center">1</td><td align="center">1</td><td align="center">1</td></tr></table>
</div>
<div style="float:left;margin:10px 50px;text-align:center;">
<table class="grid"><tr><th style="width:50px;">x</th>
<th style="width:50px;">y</th>
<th>x XOR y</th></tr>
<tr><td align="center">0</td><td align="center">0</td><td align="center">0</td></tr><tr><td align="center">0</td><td align="center">1</td><td align="center">1</td></tr><tr><td align="center">1</td><td align="center">0</td><td align="center">1</td></tr><tr><td align="center">1</td><td align="center">1</td><td align="center">0</td></tr></table>
</div>
<br clear="all">

How many $6$-input binary truth tables, $\tau$, satisfy the formula
$$\tau(a, b, c, d, e, f) \mathbin{\text{AND}} \tau(b, c, d, e, f, a \mathbin{\text{XOR}} (b \mathbin{\text{AND}} c)) = 0$$
for all $6$-bit inputs $(a, b, c, d, e, f)$?

## Description

If I have a truth table, $\tau(a, b, c, d, e, f)$, then each bit combination is associated with a value in a truth table. Since there are $2^6 = 64$ possible bit combinations there are $2^{64} = 18,446,744,073,709,551,616$ possible truth tables.

Each bit combination in $\tau(a, b, c, d, e, f)$ maps to a different bit combination in $\tau(b, c, d, e, f, a \mathbin{\text{XOR}} (b \mathbin{\text{AND}} c))$. E.g.

$$(0, 0, 0, 1, 0, 1) \rightarrow (0, 0, 1, 0, 1, 0)$$

This means that the row in the $\tau$ truth table corresponding to $(0, 0, 0, 1, 0, 1)$ and the row in the $\tau$ truth table corresponding to $(0, 0, 1, 0, 1, 0)$ cannot both be $1$, or the 

$$\tau(\ldots) \mathbin{\text{AND}} \tau(\ldots) = 0$$ 

conditions will be violated.

## Attempt 1: Recursive algorithm

The basis for the attack will be a recursive algorithm that will try to build truth tables without contradictions, and will terminate if a contraditrary state is reached. This way, I can avoid searching through all $2^{64}$ states.

In [21]:
from typing import List
from itertools import product


def bit_list_to_decimal(bits: List[int]) -> int:
    return int("".join(map(str, bits)), base=2)


# two bit case
two_bit_table_map = []
for a, b in product([0, 1], repeat=2):
    two_bit_table_map.append(bit_list_to_decimal([b, a ^ b]))

# three bit case
three_bit_table_map = []
for a, b, c in product([0, 1], repeat=3):
    three_bit_table_map.append(bit_list_to_decimal([b, c, a ^ (b and c)]))

# four bit case
four_bit_table_map = []
for a, b, c, d in product([0, 1], repeat=4):
    four_bit_table_map.append(bit_list_to_decimal([b, c, d, a ^ (b and c)]))

# five bit case
five_bit_table_map = []
for a, b, c, d, e in product([0, 1], repeat=5):
    five_bit_table_map.append(bit_list_to_decimal([b, c, d, e, a ^ (b and c)]))

# six bit case
six_bit_table_map = []
for a, b, c, d, e, f in product([0, 1], repeat=6):
    six_bit_table_map.append(bit_list_to_decimal([b, c, d, e, f, a ^ (b and c)]))

In [28]:
from typing import Tuple


def num_truth_tables(tt_map: List[int], print_output=True) -> Tuple[int, int]:
    def _num_truth_tables(truth_table: List[int] = None) -> Tuple[int, int]:
        truth_table = [] if truth_table is None else truth_table

        for i in range(len(truth_table)):
            mapped_index = tt_map[i]
            # contradiction
            if mapped_index < len(truth_table) and not (truth_table[i] & truth_table[mapped_index] == 0):
                return 0, 1

        if len(tt_map) == len(truth_table):  # stopping criterion
            return 1, 1
        else:
            left_result, left_calls = _num_truth_tables(truth_table + [0])
            right_result, right_calls = _num_truth_tables(truth_table + [1])
            return left_result + right_result, left_calls + right_calls
    
    result, num_calls = _num_truth_tables()

    if print_output:
        print("")
        print("len(tt_map): ", len(tt_map))
        print("num truth table:", 2**len(tt_map))
        print("num calls: ", num_calls)
        print("Call reduction: ", num_calls/2**len(tt_map))
        print("Result: ", result)
    return result, num_calls

In [23]:
num_truth_tables(two_bit_table_map)
num_truth_tables(three_bit_table_map)
num_truth_tables(four_bit_table_map)
num_truth_tables(five_bit_table_map)


len(tt_map):  4
num truth table: 16
num calls:  8
Call reduction:  0.5
Result:  4

len(tt_map):  8
num truth table: 256
num calls:  54
Call reduction:  0.2109375
Result:  28

len(tt_map):  16
num truth table: 65536
num calls:  3256
Call reduction:  0.0496826171875
Result:  1596

len(tt_map):  32
num truth table: 4294967296
num calls:  6254173
Call reduction:  0.0014561631251126528
Result:  2961596


(2961596, 6254173)

## Attempt 2: Loops

The recursive algorithm turns out to be much too slow because, even though I terminate the stack as soon as a contradiction is hit, the number of truth tables that have to be checked go out two orders of magnitude for each additional bit.

The basis for the second attempt is the observation that the mapping between rows in the generated truth tables give rise to "loops" where two adjacant rows in the loop cannot both be 1, or it would violate the $\tau(\ldots) \mathbin{\text{AND}} \tau(\ldots) = 0$. However, distinct loops can be handled independently of each other, meaning that the number of truth tables that have to be searched through can be drastically reduced. 

For example, for the thee-bit case, the "row map" is the following:

```
0 -> 0
1 -> 2
2 -> 4
3 -> 7
4 -> 1
5 -> 3
6 -> 5
7 -> 6
```

which means that 0 is paired with itself in the truth table, row 1 is paired with row 2, row 2 is paired with row 4, ...

This gives rise to the following three loops

$$
0 \rightarrow 0 \\
1 \rightarrow 2 \rightarrow 4 \rightarrow 1 \\
3 \rightarrow 7 \rightarrow 6 \rightarrow 5 \rightarrow 3 \\
$$

Thus, the problem can be decomposed into three subproblems, where no two adjacant elements in the loop can both be equal to 1. For the first loop ($0 \rightarrow 0$) there is just one way to do this, for the second loop there are four ways, and for the last loop there are 7 ways. So the solution becomes

$$1 \times 4 \times 7 = 28$$

This approach can also be applied to cases with more bits, and should scale well.

In [71]:
def find_loops(row_map: List[int]) -> list[list[int]]:
    loops = []
    for i in row_map:
        loop = []
        if not any([element == i for sublist in loops for element in sublist]):
            loop.append(i)
            while True:
                i = row_map[i]
                if i in loop:
                    loops.append(loop)
                    break
                else:
                    loop.append(i)
    return loops


loops = find_loops(three_bit_table_map)
for loop in loops:
    print(loop)

[0]
[2, 4, 1]
[7, 6, 5, 3]


In [81]:
import functools


@functools.cache
def valid_truth_tables(
    size: int, first_element: int = -1, latest_element: int = -1, current_size: int = -1
) -> int:
    if first_element == -1: # initiate
        return valid_truth_tables(size, 0, 0, 1) + valid_truth_tables(size, 1, 1, 1)
    if current_size == size:  # stopping criterion
        if latest_element == 1 and first_element == 1:
            return 0
        else:
            return 1
    if latest_element == 0:
        return valid_truth_tables(size, first_element, 0, current_size + 1) + valid_truth_tables(size, first_element, 1, current_size + 1)
    else:
        return valid_truth_tables(size, first_element, 0, current_size + 1)


# sanity checks
print("loop size 1: ", valid_truth_tables(1))
print("loop size 2: ", valid_truth_tables(2))
print("loop size 3: ", valid_truth_tables(3))
print("loop size 4: ", valid_truth_tables(4))

loop size 1:  1
loop size 2:  3
loop size 3:  4
loop size 4:  7


In [85]:
def num_truth_tables2(tt_map=List[int]) -> int:
    loops = find_loops(tt_map)
    num_solutions = 1
    valid_truth_tables.cache_clear()
    for loop in loops:
        num_solutions *= valid_truth_tables(len(loop))
    return num_solutions


print("2 bit = ", num_truth_tables2(two_bit_table_map))
print("3 bit = ", num_truth_tables2(three_bit_table_map))
print("4 bit = ", num_truth_tables2(four_bit_table_map))
print("5 bit = ", num_truth_tables2(five_bit_table_map))
print("6 bit = ", num_truth_tables2(six_bit_table_map))

2 bit =  4
3 bit =  28
4 bit =  1596
5 bit =  2961596
6 bit =  15964587728784
